### ベクトルストアでの利用
1. FAISSに格納
2. FAISSから取得
3. FAISSでベクトル検索

In [2]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from pypdf import PdfReader
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv('../.env')

True

#### 1. FAISSに格納

In [3]:
# 1.PDFファイルの読み込み
pdf_page = PdfReader('./data/llm.pdf')
text = ''
# PDFデータをテキストに変換
for page in pdf_page.pages:
    text += page.extract_text()
print(text[:100])  # 最初の100文字を表示

⼤規模⾔語モデル
出典 : フリー百科事典『ウィキペディア（ Wikipedia ）』
⼤規模⾔語モデル（だいきぼげんごモデル、英: large language model、LLM）は、多数のパラメ


In [7]:
# 2. データをチャンクに小分けにする
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # チャンクの最大文字数
    chunk_overlap=100, # チャンク間の重複文字数
)
docs = text_splitter.split_text(text) # テキストをチャンクに分割
docs_chunks = text_splitter.create_documents(docs) # チャンクをドキュメントに変換
print("Docサイズ",len(docs_chunks))  # チャンクの数を表示
print("型",type(docs_chunks[0]))  # チャンクの型を表示

Docサイズ 64
型 <class 'langchain_core.documents.base.Document'>


In [8]:
# 3. OpenAIのEmbeddingModel取得
embeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-3-small",
        openai_api_version="2023-05-15",
    )
else:
    print("APIKeyの設定を確認してください")

In [9]:
# 4. ベクトル化
# 一括でベクトル化する場合
# faiss_db = FAISS.from_documents(documents=docs, embedding=embedings)

# プログレスバーを表示しながらベクトル化
faiss_db = None
with tqdm(total=len(docs_chunks), desc="docments 抽出") as pbar:
    for d in docs_chunks:
        if faiss_db:
            faiss_db.add_documents([d])
        else:
            faiss_db = FAISS.from_documents([d], embeddings)
        pbar.update(1)

docments 抽出: 100%|██████████| 64/64 [00:14<00:00,  4.30it/s]


In [10]:
# 5. ベクトルストアの保存
faiss_db.save_local("./db")

#### 2. FAISSから取得

In [12]:
# ベクトル化データを読み込み
vectorstore = FAISS.load_local("./db", # ベクトル化データの保存先
embeddings, # ベクトル化モデル
allow_dangerous_deserialization=True # 危険なデシリアライズを許可
)

# retrieverを取得
retriever = vectorstore.as_retriever()

#### 3. FAISSでベクトル検索

In [14]:
# 類似文書の取得
similar_docs = retriever.get_relevant_documents("言語分析", k=3)
# similar_docs = retriever.invole("言語モデル", k=3)

# 類似文書を表示
for doc in similar_docs:
    print(doc)
    print("-----")

page_content='である。たとえば、 LLM は「 Can you teach an old dog new tricks? （年⽼いた⽝に新しい芸を教えられます
か？）」という質問に対して、「you can't teach an old dog new tricks（⽼⽝に新しい芸を仕込むことはでき
ない）」という英語の語法に触れた結果、⽂字通り真実でないにもかかわらず、「 No 」と答えるかもしれな
い[67]。
さらに、 AI が多肢選択式テスト（○ × 式テスト）において、必ずしも実際に訪ねられている設問を理解するこ
となく表⾯的な問題⽂の統計的相関を利⽤して正解を推測し、「カンニング」する「ショートカット学習」と
呼ばれるケースもある[68]。
敵対的評価データセットのもう⼀つの例は、 Swag とその後継の HellaSwag である。これは、⽂章を完成させ
るためにいくつかの選択肢から⼀つを選択しなければならない問題を集めたものである。不正解の選択肢
は、⾔語モデルからサンプリングし、⼀連の分類器でフィルタリングすることで作成された。その結果、⼈
タスク固有のデータセットとベンチマーク
逆説的に構成された評価間にとっては些細な問題でも、デ タセットが作成された当時は、最先端の⾔語モデルの精度は思わしくな
かった。たとえば、次のようなものである。
フィットネスセンターの看板が⾒える。そして、エクササイズボールに座ったり横たわりなが
ら、カメラに向かって話しかける男性が⾒える。その男性は、 ...
a) ボールの上を⾛ったり降りたりして、運動の効果を効率的にする⽅法を実演している。
b) すべての腕と脚を動かしてたくさんの筋⾁をつけている。
c) 次にボールを投げ、グラフィックや⽣け垣の刈り込みの実演を⾒る。
d) ボールの上で腹筋運動をしながら話をしている[69]。
BERT は最も可能性の⾼い補完として b) を選択したが、正解は d) である[69]。
⼤規模⾔語モデルは、それ⾃体が「ブラックボックス」であり、どのようにして⾔語タスクを実⾏できるの
かは明らかではない。しかし、 LLM がどのように機能するかを理解するためのいくつかの⽅法がある。
機械的解釈可能性  は、 LLM によって実⾏される推論を近似する記号アルゴリズムを発⾒するこ